In [3]:
'''
Training subsample prototypes
'''
import numpy as np
import pandas as pd
from sklearn.utils.class_weight import compute_sample_weight

demo_tgt = np.array([
    90,90,90,42,42,42,90,62,62,90,42,67,42,90,90,42,15,15,6,90
])
pd.Series(demo_tgt).value_counts()

90    8
42    6
62    2
15    2
6     1
67    1
dtype: int64

In [16]:
sw = compute_sample_weight('balanced', demo_tgt, indices=demo_tgt>60)

In [17]:
pd.DataFrame(np.vstack([demo_tgt, sw]).T).groupby(0)[1].sum()

0
6.0     0.000000
15.0    0.000000
42.0    0.000000
62.0    3.666667
67.0    3.666667
90.0    3.666667
Name: 1, dtype: float64

In [29]:
md = {90 : (4,0), 42 : (2,1), 15 : (2,1)}

def mask_samples(tgt, mask_dict):
    '''
    Compute a boolean mask to cover tgt classes as specified by mask_dict.
    Mask_dict entries as follows:
    k : v, where k is the tgt class to mask, and v is a (v0, v1) tuple, where v0 is the number of splits to
    make in class k, and v1 is the split number to leave unmasked (ie. True). All other splits are masked
    '''
    
    # Full mask
    mask = np.ones(tgt.size)
    
    for k,v in mask_dict.items():
        # Get tgt indices
        k_ixs = np.nonzero(tgt==k)[0]
        # Split tgt indices
        split_k_ixs = np.array_split(k_ixs, v[0])
        
        # Cover all k splits except the choosen split
        for i,s in enumerate(split_k_ixs):
            if i != v[1]:
                mask[s] = 0
                
    return mask.astype(bool)
    
msk = mask_samples(demo_tgt, md)
msk

array([ True,  True, False, False, False, False, False,  True,  True,
       False,  True,  True,  True, False, False,  True, False,  True,
        True, False])

In [41]:
sw = compute_sample_weight('balanced', demo_tgt[msk])

In [42]:
sw

array([0.83333333, 0.83333333, 0.83333333, 0.83333333, 0.55555556,
       1.66666667, 0.55555556, 0.55555556, 1.66666667, 1.66666667])

In [38]:
sw

array([0.83333333, 0.83333333, 0.83333333, 0.55555556, 0.55555556,
       0.55555556, 0.83333333, 0.83333333, 0.83333333, 0.83333333,
       0.55555556, 1.66666667, 0.55555556, 0.83333333, 0.83333333,
       0.55555556, 1.66666667, 1.66666667, 1.66666667, 0.83333333])